## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


## Практическая часть


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys


In [2]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [4]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [6]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [7]:
df = pd.DataFrame()

In [8]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [9]:
for factor in [30, 50, 80, 100, 300, 500]:
    for reg in [0.0005, 0.001, 0.005, 0.01, 0.05]:
        #model = ALS(factor, ...)
        model = AlternatingLeastSquares(factors=44, 
                                    regularization=reg,
                                    iterations=15, 
                                    calculate_training_loss=True, 
                                    use_gpu=False)

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

        recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)

        result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        
        Score = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
        name = 'factor = ' + str(factor) + ' regularization = ' + str(reg)
        new_row = {'name':name, 'Score':Score}
        df = df.append(new_row, ignore_index=True)
        
        print (name + '\nprecision_at_k = ' + str(Score))
        #result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

100%|██████████| 15/15 [00:02<00:00,  5.16it/s, loss=0.0484]
C:\Users\111\AppData\Local\Temp\ipykernel_13180\2015075604.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


factor = 30 regularization = 0.0005
precision_at_k = 0.18570029382957887


100%|██████████| 15/15 [00:03<00:00,  4.67it/s, loss=0.0485]
C:\Users\111\AppData\Local\Temp\ipykernel_13180\2015075604.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


factor = 30 regularization = 0.001
precision_at_k = 0.18305582761998043


100%|██████████| 15/15 [00:02<00:00,  5.49it/s, loss=0.0491]
C:\Users\111\AppData\Local\Temp\ipykernel_13180\2015075604.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


...

100%|██████████| 15/15 [00:02<00:00,  5.25it/s, loss=0.0494]
C:\Users\111\AppData\Local\Temp\ipykernel_13180\2015075604.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


factor = 500 regularization = 0.01
precision_at_k = 0.18893241919686585


100%|██████████| 15/15 [00:02<00:00,  5.37it/s, loss=0.0499]


factor = 500 regularization = 0.05
precision_at_k = 0.18893241919686585


C:\Users\111\AppData\Local\Temp\ipykernel_13180\2015075604.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [10]:
df

,name,Score
0,factor = 30 regularization = 0.0005,0.185700
1,factor = 30 regularization = 0.001,0.183056
2,factor = 30 regularization = 0.005,0.186582
3,factor = 30 regularization = 0.01,0.188345
4,factor = 30 regularization = 0.05,0.188737
5,factor = 50 regularization = 0.0005,0.184623
6,factor = 50 regularization = 0.001,0.186974
7,factor = 50 regularization = 0.005,0.188443
8,factor = 50 regularization = 0.01,0.186190
9,factor = 50 regularization = 0.05,0.185700


In [11]:
df['Score'].max()

0.19157688540646428

Выводы:
1. Увеличение факторов не ведет к увеличению качества. Перед началом работы слеует провести анализ данных и попытатся установить оптимальное количество факторов. В примере из леции - это производители (department). В нашей выборке - опереться не на что, поэтому надо подбирать. Попробовала как в маленьком диапазоне. Так и сильно увеличить. Собственно, как и ожидала - сильное увеличние факторов не ведет к увеличению качества. А вот привести к переобучению модели вполне может, как я считаю.
2. Настройки regularization уже более тонки и шаг тут уже меньше. Но его эксримально низким делать не стоит. Качество не увеличвается, а вот время обработки растет. Про время обработки - это по ущещниям. Не замеряла, но в рамках обучения так и было.
3. По итогу, лучшая модель - на факторах 80 и regularization 0.05